https://github.com/ibm-watson-data-lab/pixiedust/wiki/Tutorial:-Using-Notebooks-with-PixieDust-for-Fast,-Flexible,-and-Easier-Data-Analysis-and-Experimentation

In [3]:
#!pip install --user -U pixiedust requests matplotlib
import pixiedust

inspections = pixiedust.sampleData("https://opendata.lasvegasnevada.gov/resource/86jg-3buh.csv")

Downloaded 364109 bytes
Creating pandas DataFrame for 'https://opendata.lasvegasnevada.gov/resource/86jg-3buh.csv'. Please wait...
Loading file using 'pandas'
Successfully created pandas DataFrame for 'https://opendata.lasvegasnevada.gov/resource/86jg-3buh.csv'


In [4]:
display(inspections)

:@computed_region_tnyv_z3b7,address,category_name,city,current_demerits,current_grade,date_current,employee_id,inspection_date,inspection_demerits,inspection_grade,inspection_result,inspection_time,inspection_type,location_1,location_1_address,location_1_city,location_1_state,location_1_zip,location_name,permit_number,permit_status,record_updated,restaurant_name,serial_number,state,violations,zip
nan,361 Symphony Park Ave,Bar / Tavern,Las Vegas,0,A,2017-08-13T00:00:00.000,EE7000806,2017-08-13T00:00:00.000,0,A,'A' Grade,2017-08-13T12:45:00.000,Routine Inspection,POINT (115.1528009 36.1684496),nan,nan,nan,nan,SMITH PERFORMING CENTER - REYNOLDS,PR0021214,A,2017-08-15T11:26:19.000,REYNOLDS 5TH FLOOR WEST BAR,DA1801669,Nevada,nan,89102
nan,3377 S Las Vegas Blvd 2610,Restaurant,Las Vegas,0,A,2018-01-17T00:00:00.000,EE7001288,2017-08-10T00:00:00.000,0,A,'A' Grade,2017-08-10T13:25:00.000,Routine Inspection,POINT (115.1696491 36.1218632),nan,nan,nan,nan,WASABI JANES @ VENETIAN,PR0016176,A,2017-08-10T14:09:27.000,WASABI JANES RESTAURANT,DA9MLW6B8,Nevada,2930,89109-8910
nan,3327 S Las Vegas Blvd 2760,Bar / Tavern,Las Vegas,0,A,2018-02-04T00:00:00.000,EE7001184,2017-08-14T00:00:00.000,0,A,'A' Grade,2017-08-14T18:30:00.000,Routine Inspection,POINT (115.167872 36.1240615),nan,nan,nan,nan,LAVO CAFE @ PALAZZO,PR0020511,A,2017-08-14T18:23:02.000,LAVO CAFE CLUB LEVEL BAR,DAZVWLNGE,Nevada,nan,89109-1402
nan,3663 S LAS VEGAS Blvd 410,Restaurant,Las Vegas,9,A,2017-12-11T00:00:00.000,EE7001217,2017-08-06T00:00:00.000,42,X,Closed with Fees,2017-08-06T15:30:00.000,Routine Inspection,POINT (115.0599337 36.2376578),nan,nan,nan,nan,KETCHUP @ MIRACLE MILE,PR0107255,CLOSED,2017-08-06T20:12:23.000,KETCHUP RESTAURANT,DAEQ4X8WF,Nevada,"202,208,209,211,213,216,217,218,305,2909,2910,2928,2929,2930,2955,2956",89101
nan,4835 W Craig Rd 3-D,Restaurant,Las Vegas,8,A,2017-08-15T00:00:00.000,EE7001215,2017-08-15T00:00:00.000,8,A,'A' Grade,2017-08-15T09:20:00.000,Routine Inspection,POINT (115.208853 36.23855),nan,nan,nan,nan,Harvest Cafe,PR0020078,A,2017-08-15T10:36:08.000,Harvest Cafe,DAFCX9BAV,Nevada,"206,2909,2912,2930",89130-2728
nan,8556 BLUE DIAMOND Rd,Restaurant,Las Vegas,3,A,2017-08-09T00:00:00.000,EE7000594,2017-08-09T00:00:00.000,3,A,'A' Grade,2017-08-09T08:00:00.000,Routine Inspection,POINT (115.2778999 36.0208765),nan,nan,nan,nan,MCDONALDS USA LLC,PR0104534,A,2017-08-09T08:46:05.000,MCDONALD'S,DAEB1WES6,Nevada,222,89178
nan,3121 S Las Vegas Blvd,Garde Manger,Las Vegas,0,A,2017-08-14T00:00:00.000,EE7000366,2017-08-14T00:00:00.000,0,A,'A' Grade,2017-08-14T13:40:00.000,Routine Inspection,POINT (115.1652708 36.1293177),nan,nan,nan,nan,ENCORE HOTEL & CASINO,PR0013053,A,2017-08-14T13:51:38.000,ENCORE GARDE MANGER PLATING ROOM,DAIZYR6IA,Nevada,nan,89109-1967
nan,3743 S LAS VEGAS,Restaurant,Las Vegas,3,A,2017-08-14T00:00:00.000,EE7001290,2017-07-31T00:00:00.000,25,C,'C' Downgrade,2017-07-31T15:15:00.000,Routine Inspection,POINT (115.1720033 36.1065567),nan,nan,nan,nan,PAPAYA KING @ HAWAIIAN MARKETPLACE,PR0121745,C,2017-07-31T17:34:38.000,PAPAYA KING,DAQ1FKRSI,Nevada,"206,209,211,215,218,2926,2928,2929,2930,2955,2956",89109
nan,4663 E Sunset Rd,Restaurant,Henderson,0,A,2017-08-10T00:00:00.000,EE7001226,2017-02-28T00:00:00.000,6,A,'A' Grade,2017-02-28T11:35:00.000,Routine Inspection,POINT (115.072089 36.070822),nan,nan,nan,nan,Fatburger #126,PR0024379,A,2017-02-28T12:36:15.000,Fatburger #126,DA1ISTLZU,Nevada,"211,216,2929,2930",89014-2215
nan,121 CARNEGIE St,Restaurant,Henderson,6,A,2017-08-14T00:00:00.000,EE7000772,2017-08-14T00:00:00.000,6,A,'A' Grade,2017-08-14T13:45:00.000,Routine Inspection,POINT (115.0913627 36.0178059),nan,nan,nan,nan,LIFE TIME ATHLETIC,PR0107493,A,2017-08-14T13:30:33.000,LIFE TIME ATHLETIC - POOLSIDE BISTRO,DAP8QCYDC,Nevada,"211,218",89074


# Using pymongo and pixiedust

our server at 10.210.9.130 uses ssl with a self-signed certificate


In [ ]:
# install pymongo
#!pip install --user -U pymongo

In [1]:
#!conda install -y pymongo
import pymongo as pm
from pymongo.errors import ConnectionFailure
from time import sleep
from ssl import CERT_NONE


client = pm.MongoClient('10.210.9.130', ssl=True, ssl_cert_reqs=CERT_NONE,
                        username='strands', authSource='admin', authMechanism='SCRAM-SHA-1',
                        serverSelectionTimeoutMS=5000, password='Grum#Beere')
#client = pm.MongoClient('lcas.lincoln.ac.uk', port=80)
#try:
   # The ismaster command is cheap and does not require auth.


collection = client.aafy4_message_store.nav_stats

c= collection.find({}).limit(100000)

import pandas as pd
df = pd.DataFrame(list(c))
print(df.columns)
print(df.origin.describe())
print(df.describe())

from scipy import stats
stats.normaltest(df.time_to_waypoint)
stats.spearmanr(df.time_to_waypoint, df.operation_time)


contingency = pd.crosstab(df.origin, df.status)
#print(contingency)
#c, p, dof, expected = stats.chi2_contingency(contingency)
#stats.chi2_contingency(contingency)


Index(['_id', '_meta', 'date_at_node', 'date_finished', 'date_started',
       'edge_id', 'final_node', 'mon_events', 'operation_time', 'origin',
       'status', 'target', 'time_to_waypoint', 'topological_map'],
      dtype='object')
count       32959
unique         81
top       Frisoer
freq         1876
Name: origin, dtype: object
       operation_time  time_to_waypoint
count    32959.000000      32959.000000
mean        13.180939          1.139454
std         21.725064          7.077456
min          0.114698          0.000000
25%          4.993971          0.000000
50%          7.834488          0.139613
75%         13.397786          0.200584
max        620.919739        442.967409


In [2]:
import pixiedust

display(df)
#print(df)


# Use rosbag with pixiedust

* needs to install ROS and somehow make it work with jupyter (loadenv)
* then needs to use rosbag_pandas
* then we are happy and can go

In [ ]:
# You can use python inline. E.g. you can define functions to be used later:

# import environment and expose in jupyter env
# this only works when shellvars is installed as it is in L-CAS Juptyer images
# https://github.com/LCAS/jupyterhub-deploy-docker/blob/master/singleuser/Dockerfile#L100

import shellvars
from os import environ

def loadenv(filename='/opt/ros/kinetic/setup.bash'):
    vs = shellvars.get_vars(filename)
    for v in vs:
        print('%s=%s' % (v.decode("utf-8"), str(vs[v].decode("utf-8"))))
        environ[v.decode("utf-8")] = str(vs[v].decode("utf-8"))

# you can call these functions easily
# (This example is a good way to load environment variables from a config file)

loadenv('/opt/ros/kinetic/setup.bash')

In [ ]:

# install marc-hanheide's own python3 version of rosbag_panda: https://github.com/marc-hanheide/RosbagPandas
#   - clone it
#   - run pip install --user -U . in it 
# tutorial at https://nimbus.unl.edu/2014/11/using-rosbag_pandas-to-analyze-rosbag-files/
#sudo apt-get install ros-kinetic-rosbag python-roslz4
def expand_sys_path():
    import sys
    from os import environ

    extra_paths = environ['PYTHONPATH'].split(':')
    sys.path.extend(extra_paths)

    new_path = []
    for p in sys.path:
        if not p in new_path:
            new_path.append(p)

    sys.path = new_path


expand_sys_path()

import rosbag
import pixiedust
from pprint import pprint
bag = rosbag.Bag('bag.bag')
topics = bag.get_type_and_topic_info()[1]

for t in topics:
    print(t, topics[t])

import rosbag_pandas

df = rosbag_pandas.bag_to_dataframe('bag.bag', include=['/amcl_pose'])



In [ ]:
display(df)